# Introduction 

The below script loads data from Calls_for_service file and merges with data from max_cfs_ucr_categories file.
Following constraints are applied:
- Records year range = 2012 - 2016
- Records Disposition = RTF 
- Records CrimeType = Violent Crime

# Index

- [1. Libraries](#1.-Libraries)
- [2. Data Loading](#2.-Data-Loading)
    - [A. Load Calls-for-Service (CFS) Data](#A.-Load-Calls-for-Service-Data)
        - [i. Filter Records with Disposition 'RTF'](#i.-Filter-Records-with-Disposition-'RTF')
        - [ii. Filter Records for Year 2012 to 2016](#ii.-Filter-Records-for-Year-2012-to-2016)    
    - [B. Load Categories Data](#B.-Load-Categories-Data)
        - [i. Filter Records for CrimeType 'Violent Crime'](#i.-Filter-Records-for-CrimeType-'Violent-Crime')
- [3. Merge CFS with Categories data](#3.-Merge-CFS-with-Categories-data)
- [4. Verify Records](#4.-Verify-Records)
- [5. Generate Lat/Long from Location](#5.-Generate-Lat/Long-from-Location)
- [6. Check Beat Values](#6.-Check-Beat-Values)
    - [A. Load missing Beat Values](#A.-Load-missing-Beat-Values)
- [7. Save CFS Data to csv](#7.-Save-CFS-Data-to-csv)
- [8. Zip with most Violent Crimes](#8.-Zip-with-most-Violent-Crimes)


# 1. Libraries

Following libraries are required for this code to run successfully.

In [1]:
# Load required libraries 
import os
import csv
import zipfile
import string
import pandas as pd
import datetime as dt
from collections import Counter

# 2. Data Loading

In this section we load following files:
- Calls_for_Service Data
- MAX_CFS_UCR_Categories Data

And perform required data manipulation steps.

## A. Load Calls-for-Service Data

In [2]:
# Set location of file
path = os.path.join("../Datasets/Raw_Data/Calls_for_Service/")
path

'../Datasets/Raw_Data/Calls_for_Service/'

In [3]:
# Get filenames
filenames = os.listdir(path)
filenames

['Calls_for_Service_2012.zip',
 'Calls_for_Service_2013.zip',
 'Calls_for_Service_2014.zip',
 'Calls_for_Service_2015.zip',
 'Calls_for_Service_2016.zip']

In [4]:
# Load data from files in list
dfs = []
for f in filenames:
    zf = zipfile.ZipFile(os.path.join(path,f)) 
    dfs.append(pd.read_csv(zf.open(str.replace(f, 'zip', 'csv')),))

# Merge all df in list
cfs_df = pd.concat(dfs, ignore_index=True)

# Change datatype of column Type_ to String
cfs_df.Type_ = cfs_df.Type_.apply(str)

In [5]:
# Display top 5 rows
cfs_df.head()

,NOPD_Item,Type_,TypeText,Priority,InitialType,InitialTypeText,InitialPriority,MapX,MapY,TimeCreate,...,TimeArrive,TimeClosed,Disposition,DispositionText,SelfInitiated,Beat,BLOCK_ADDRESS,Zip,PoliceDistrict,Location
0,A0000112,62A,"BURGLAR ALARM, SILEN",2C,NaN,NaN,NaN,3683627,532625,1/1/2012 0:00,...,NaN,1/1/2012 0:33,NAT,NECESSARY ACTION TAKEN,NaN,NaN,009XX Decatur St,70116.0,8,"(29.958469303316875, -90.0613152964016)"
1,A0000412,94,DISCHARGING FIREARMS,2B,NaN,NaN,NaN,3732996,562418,1/1/2012 0:00,...,1/1/2012 0:16,1/1/2012 0:30,UNF,UNFOUNDED,NaN,NaN,147XX Chef Menteur Hwy,70129.0,7,"(30.038788769111676, -89.90425047516077)"
2,A0000212,103,DISTURBANCE (OTHER),1C,NaN,NaN,NaN,3687688,548824,1/1/2012 0:01,...,1/1/2012 0:01,1/1/2012 0:19,NAT,NECESSARY ACTION TAKEN,NaN,NaN,038XX Gentilly Blvd,70122.0,3,"(30.002886229898206, -90.04791794333323)"
3,A0000712,21,COMPLAINT OTHER,1H,NaN,NaN,NaN,3670776,521242,1/1/2012 0:01,...,NaN,1/1/2012 0:20,NAT,NECESSARY ACTION TAKEN,NaN,NaN,Carondelet St & Napoleon Ave,70115.0,2,"(29.927555772946167, -90.10228161624175)"
4,A0000512,62A,"BURGLAR ALARM, SILEN",2C,NaN,NaN,NaN,3665739,549621,1/1/2012 0:01,...,1/1/2012 0:09,1/1/2012 1:55,NAT,NECESSARY ACTION TAKEN,NaN,NaN,002XX W Harrison Ave,70124.0,3,"(30.005736477457617, -90.11723146931276)"


In [6]:
# Number of records
len(cfs_df)

2252907

In [7]:
# Column Names
cfs_df.columns

Index([u'NOPD_Item', u'Type_', u'TypeText', u'Priority', u'InitialType',
       u'InitialTypeText', u'InitialPriority', u'MapX', u'MapY', u'TimeCreate',
       u'TimeDispatch', u'TimeArrive', u'TimeClosed', u'Disposition',
       u'DispositionText', u'SelfInitiated', u'Beat', u'BLOCK_ADDRESS', u'Zip',
       u'PoliceDistrict', u'Location'],
      dtype='object')

### i. Filter Records with Disposition 'RTF' 

In [8]:
cfs_df = cfs_df[cfs_df.Disposition.str.strip() == 'RTF']

In [9]:
# Number of records
len(cfs_df)

428570

In [10]:
# Display top 5 rows
cfs_df.head()

,NOPD_Item,Type_,TypeText,Priority,InitialType,InitialTypeText,InitialPriority,MapX,MapY,TimeCreate,...,TimeArrive,TimeClosed,Disposition,DispositionText,SelfInitiated,Beat,BLOCK_ADDRESS,Zip,PoliceDistrict,Location
64,A0006912,29S,SUICIDE,2B,NaN,NaN,NaN,37369000,3513814,1/1/2012 0:16,...,1/1/2012 0:32,1/1/2012 0:59,RTF,REPORT TO FOLLOW,NaN,NaN,049XX Major Dr,70128.0,0,"(9.235500542976004E-7, -0.000002292984766499941)"
79,A0008112,966,DRUG VIOLATIONS,1G,NaN,NaN,NaN,3680920,540222,1/1/2012 0:22,...,1/1/2012 0:23,1/1/2012 1:13,RTF,REPORT TO FOLLOW,NaN,NaN,St Bernard Ave & N Tonti St,70119.0,1,"(29.979440793120016, -90.06959870234981)"
80,A0008212,21U,UNDER AGE DRINKING V,1F,NaN,NaN,NaN,3681403,531510,1/1/2012 0:22,...,1/1/2012 0:22,1/1/2012 1:33,RTF,REPORT TO FOLLOW,NaN,NaN,003XX Bourbon St,70112.0,8,"(29.95547113098435, -90.06837696099961)"
83,A0008312,66,EXTORTION (THREATS),1F,NaN,NaN,NaN,3691753,536508,1/1/2012 0:23,...,1/1/2012 0:25,1/1/2012 1:19,RTF,REPORT TO FOLLOW,NaN,NaN,014XX Alvar St,70117.0,5,"(29.96889564722802, -90.03551677167968)"
92,A0009412,94,DISCHARGING FIREARMS,2B,NaN,NaN,NaN,3696950,533169,1/1/2012 0:28,...,1/1/2012 0:28,1/1/2012 1:36,RTF,REPORT TO FOLLOW,NaN,NaN,054XX Burgundy St,70117.0,5,"(29.959551708547103, -90.01922533580768)"


### ii. Filter Records for Year 2012 to 2016

In [11]:
# Convert column type to datetime
cfs_df.TimeCreate = pd.to_datetime(cfs_df.TimeCreate)

In [12]:
# Apply filter for year range 2014-2016
cfs_df = cfs_df[(cfs_df.TimeCreate >= dt.date(2012,1,1)) \
                & (cfs_df.TimeCreate < dt.date(2017,1,1))]

In [13]:
len(cfs_df)

428570

In [14]:
# Display top 5 rows
cfs_df.head()

,NOPD_Item,Type_,TypeText,Priority,InitialType,InitialTypeText,InitialPriority,MapX,MapY,TimeCreate,...,TimeArrive,TimeClosed,Disposition,DispositionText,SelfInitiated,Beat,BLOCK_ADDRESS,Zip,PoliceDistrict,Location
64,A0006912,29S,SUICIDE,2B,NaN,NaN,NaN,37369000,3513814,2012-01-01 00:16:00,...,1/1/2012 0:32,1/1/2012 0:59,RTF,REPORT TO FOLLOW,NaN,NaN,049XX Major Dr,70128.0,0,"(9.235500542976004E-7, -0.000002292984766499941)"
79,A0008112,966,DRUG VIOLATIONS,1G,NaN,NaN,NaN,3680920,540222,2012-01-01 00:22:00,...,1/1/2012 0:23,1/1/2012 1:13,RTF,REPORT TO FOLLOW,NaN,NaN,St Bernard Ave & N Tonti St,70119.0,1,"(29.979440793120016, -90.06959870234981)"
80,A0008212,21U,UNDER AGE DRINKING V,1F,NaN,NaN,NaN,3681403,531510,2012-01-01 00:22:00,...,1/1/2012 0:22,1/1/2012 1:33,RTF,REPORT TO FOLLOW,NaN,NaN,003XX Bourbon St,70112.0,8,"(29.95547113098435, -90.06837696099961)"
83,A0008312,66,EXTORTION (THREATS),1F,NaN,NaN,NaN,3691753,536508,2012-01-01 00:23:00,...,1/1/2012 0:25,1/1/2012 1:19,RTF,REPORT TO FOLLOW,NaN,NaN,014XX Alvar St,70117.0,5,"(29.96889564722802, -90.03551677167968)"
92,A0009412,94,DISCHARGING FIREARMS,2B,NaN,NaN,NaN,3696950,533169,2012-01-01 00:28:00,...,1/1/2012 0:28,1/1/2012 1:36,RTF,REPORT TO FOLLOW,NaN,NaN,054XX Burgundy St,70117.0,5,"(29.959551708547103, -90.01922533580768)"


## B. Load Categories Data

In [15]:
# Set location of file
fname = "../Datasets/Raw_Data/Shared_by_NOPD/MAX_CFS_UCR_Categories.xlsx"

# Load file
crime_types = pd.read_excel(fname,sheetname='Sheet1')

# Select required columns
crime_types = crime_types.ix[:,['Code','UCR MAIN','Description']]

# Rename columns
crime_types.rename(columns={'Code':'Type_','UCR MAIN':'CrimeType','Description':'Description'},inplace=True)

# Change datatype of column Type_ to String
crime_types.Type_ = crime_types.Type_.apply(str)
crime_types.Type_ = crime_types.Type_.str.strip()

### i. Filter Records for CrimeType 'Violent Crime'

In [16]:
crime_types = crime_types[crime_types.CrimeType.str.strip() == 'VIOLENT CRIME']

In [17]:
crime_types.head()

,Type_,CrimeType,Description
71,30,VIOLENT CRIME,HOMICIDE
72,30C,VIOLENT CRIME,HOMICIDE BY CUTTING
73,30D,VIOLENT CRIME,HOMICIDE DOMESTIC
74,30S,VIOLENT CRIME,HOMICIDE BY SHOOTING
75,34,VIOLENT CRIME,AGGRAVATED BATTERY


# 3. Merge CFS with Categories data

In [18]:
merged_df = pd.merge(cfs_df,crime_types,on='Type_',how='inner')

In [19]:
merged_df.head()

,NOPD_Item,Type_,TypeText,Priority,InitialType,InitialTypeText,InitialPriority,MapX,MapY,TimeCreate,...,Disposition,DispositionText,SelfInitiated,Beat,BLOCK_ADDRESS,Zip,PoliceDistrict,Location,CrimeType,Description
0,A0026112,65,SIMPLE ROBBERY,1B,NaN,NaN,NaN,3680346,531912,2012-01-01 01:44:00,...,RTF,REPORT TO FOLLOW,NaN,NaN,002XX N Rampart St,70112.0,8,"(29.956608537919635, -90.07170060793311)",VIOLENT CRIME,SIMPLE ROBBERY
1,A0139412,65,SIMPLE ROBBERY,1B,NaN,NaN,NaN,3682307,529905,2012-01-01 20:04:00,...,RTF,REPORT TO FOLLOW,NaN,NaN,003XX Canal St,70130.0,8,"(29.951030452997887, -90.06557853022184)",VIOLENT CRIME,SIMPLE ROBBERY
2,A0163512,65,SIMPLE ROBBERY,2B,NaN,NaN,NaN,3682519,532977,2012-01-02 00:13:00,...,RTF,REPORT TO FOLLOW,NaN,NaN,008XX Bourbon St,70116.0,8,"(29.9594709577918, -90.06480175675452)",VIOLENT CRIME,SIMPLE ROBBERY
3,A0195712,65,SIMPLE ROBBERY,1B,NaN,NaN,NaN,3669570,518651,2012-01-02 09:12:00,...,RTF,REPORT TO FOLLOW,NaN,NaN,009XX Bordeaux St,70115.0,2,"(29.92046691792707, -90.1061762712039)",VIOLENT CRIME,SIMPLE ROBBERY
4,A0248912,65,SIMPLE ROBBERY,1B,NaN,NaN,NaN,3692359,543219,2012-01-02 17:25:00,...,RTF,REPORT TO FOLLOW,NaN,NaN,030XX Morrice Duncan Dr,70126.0,5,"(29.987329613283705, -90.03336254702)",VIOLENT CRIME,SIMPLE ROBBERY


In [20]:
len(merged_df)

13954

# 4. Verify Records

In [21]:
# Check length of records with CrimeType Null
len(merged_df[merged_df.CrimeType.isnull()])

0

In [22]:
# Check length of records with CrimeType Not Null
len(merged_df[merged_df.CrimeType.notnull()])

13954

In [23]:
# Remove records with CrimeType Null
cfs_final = merged_df[merged_df.CrimeType.notnull()]

In [24]:
cfs_final.head()

,NOPD_Item,Type_,TypeText,Priority,InitialType,InitialTypeText,InitialPriority,MapX,MapY,TimeCreate,...,Disposition,DispositionText,SelfInitiated,Beat,BLOCK_ADDRESS,Zip,PoliceDistrict,Location,CrimeType,Description
0,A0026112,65,SIMPLE ROBBERY,1B,NaN,NaN,NaN,3680346,531912,2012-01-01 01:44:00,...,RTF,REPORT TO FOLLOW,NaN,NaN,002XX N Rampart St,70112.0,8,"(29.956608537919635, -90.07170060793311)",VIOLENT CRIME,SIMPLE ROBBERY
1,A0139412,65,SIMPLE ROBBERY,1B,NaN,NaN,NaN,3682307,529905,2012-01-01 20:04:00,...,RTF,REPORT TO FOLLOW,NaN,NaN,003XX Canal St,70130.0,8,"(29.951030452997887, -90.06557853022184)",VIOLENT CRIME,SIMPLE ROBBERY
2,A0163512,65,SIMPLE ROBBERY,2B,NaN,NaN,NaN,3682519,532977,2012-01-02 00:13:00,...,RTF,REPORT TO FOLLOW,NaN,NaN,008XX Bourbon St,70116.0,8,"(29.9594709577918, -90.06480175675452)",VIOLENT CRIME,SIMPLE ROBBERY
3,A0195712,65,SIMPLE ROBBERY,1B,NaN,NaN,NaN,3669570,518651,2012-01-02 09:12:00,...,RTF,REPORT TO FOLLOW,NaN,NaN,009XX Bordeaux St,70115.0,2,"(29.92046691792707, -90.1061762712039)",VIOLENT CRIME,SIMPLE ROBBERY
4,A0248912,65,SIMPLE ROBBERY,1B,NaN,NaN,NaN,3692359,543219,2012-01-02 17:25:00,...,RTF,REPORT TO FOLLOW,NaN,NaN,030XX Morrice Duncan Dr,70126.0,5,"(29.987329613283705, -90.03336254702)",VIOLENT CRIME,SIMPLE ROBBERY


In [25]:
cfs_final.tail()

,NOPD_Item,Type_,TypeText,Priority,InitialType,InitialTypeText,InitialPriority,MapX,MapY,TimeCreate,...,Disposition,DispositionText,SelfInitiated,Beat,BLOCK_ADDRESS,Zip,PoliceDistrict,Location,CrimeType,Description
13949,J1221215,42B,ORAL SEXUAL BATTERY,2A,42B,ORAL SEXUAL BATTERY,1A,37369000,3513814,2015-10-10 16:58:22,...,RTF,REPORT TO FOLLOW,N,4C03,Amazon St & Kansas St,70114.0,4,"(0, 0)",VIOLENT CRIME,ORAL SEXUAL BATTERY
13950,K0073015,42B,ORAL SEXUAL BATTERY,2A,21,COMPLAINT OTHER,2A,37369000,3513814,2015-11-01 15:14:57,...,RTF,REPORT TO FOLLOW,N,3U02,050XX Chef Menteur Hwy,70126.0,3,"(0, 0)",VIOLENT CRIME,ORAL SEXUAL BATTERY
13951,K3038915,42B,ORAL SEXUAL BATTERY,1A,42B,ORAL SEXUAL BATTERY,1A,37369000,3513814,2015-11-26 17:29:18,...,RTF,REPORT TO FOLLOW,N,3I01,008XX Louque Pl,70124.0,3,"(0, 0)",VIOLENT CRIME,ORAL SEXUAL BATTERY
13952,C0577416,42B,ORAL SEXUAL BATTERY,2A,21,COMPLAINT OTHER,1H,37369000,3513814,2016-03-05 15:46:45,...,RTF,REPORT TO FOLLOW,N,3M03,015XX Milton St,70122.0,3,"(0, 0)",VIOLENT CRIME,ORAL SEXUAL BATTERY
13953,C1635316,42B,ORAL SEXUAL BATTERY,2A,21,COMPLAINT OTHER,1H,37369000,3513814,2016-03-14 13:12:14,...,RTF,REPORT TO FOLLOW,N,3S02,020XX Lakeshore Dr,70122.0,3,"(0, 0)",VIOLENT CRIME,ORAL SEXUAL BATTERY


In [26]:
cfs_final.Type_.unique()

array(['65', '64G', '34C', '64', '55', '30S', '37', '42', '34S', '43',
       '65J', '30', '37D', '34', '64J', '64K', '34D', '43M', '30C', '42M',
       '30D', '42B'], dtype=object)

In [27]:
len(cfs_final)

13954

In [28]:
min(cfs_final.TimeCreate)

Timestamp('2012-01-01 01:44:00')

In [29]:
max(cfs_final.TimeCreate)

Timestamp('2016-12-31 19:46:46')

# 5. Generate Lat/Long from Location

In [30]:
len(cfs_final[cfs_final.Location=='(0, 0)'])

1084

Remove records with (lat,lon) = (0,0)

In [31]:
# cfs_final = cfs_final[cfs_final.Location!='(0, 0)']

In [32]:
# len(cfs_final)

Remove records with (lat,lon) = null

In [33]:
len(cfs_final[cfs_final.Location.isnull()])

27

In [34]:
cfs_final = cfs_final[cfs_final.Location.notnull()]

In [35]:
len(cfs_final)

13927

In [36]:
cfs_final['Latitude'] = [str(val).split(',')[0][1:].strip() for val in cfs_final.Location] 

In [37]:
cfs_final['Longitude'] = [str(val).split(',')[1][:-1].strip() for val in cfs_final.Location] 

In [38]:
cfs_final.reset_index(inplace=True, drop=True)

In [39]:
cfs_final.columns

Index([u'NOPD_Item', u'Type_', u'TypeText', u'Priority', u'InitialType',
       u'InitialTypeText', u'InitialPriority', u'MapX', u'MapY', u'TimeCreate',
       u'TimeDispatch', u'TimeArrive', u'TimeClosed', u'Disposition',
       u'DispositionText', u'SelfInitiated', u'Beat', u'BLOCK_ADDRESS', u'Zip',
       u'PoliceDistrict', u'Location', u'CrimeType', u'Description',
       u'Latitude', u'Longitude'],
      dtype='object')

In [40]:
cfs_final.head()

,NOPD_Item,Type_,TypeText,Priority,InitialType,InitialTypeText,InitialPriority,MapX,MapY,TimeCreate,...,SelfInitiated,Beat,BLOCK_ADDRESS,Zip,PoliceDistrict,Location,CrimeType,Description,Latitude,Longitude
0,A0026112,65,SIMPLE ROBBERY,1B,NaN,NaN,NaN,3680346,531912,2012-01-01 01:44:00,...,NaN,NaN,002XX N Rampart St,70112.0,8,"(29.956608537919635, -90.07170060793311)",VIOLENT CRIME,SIMPLE ROBBERY,29.956608537919635,-90.07170060793311
1,A0139412,65,SIMPLE ROBBERY,1B,NaN,NaN,NaN,3682307,529905,2012-01-01 20:04:00,...,NaN,NaN,003XX Canal St,70130.0,8,"(29.951030452997887, -90.06557853022184)",VIOLENT CRIME,SIMPLE ROBBERY,29.951030452997887,-90.06557853022184
2,A0163512,65,SIMPLE ROBBERY,2B,NaN,NaN,NaN,3682519,532977,2012-01-02 00:13:00,...,NaN,NaN,008XX Bourbon St,70116.0,8,"(29.9594709577918, -90.06480175675452)",VIOLENT CRIME,SIMPLE ROBBERY,29.9594709577918,-90.06480175675452
3,A0195712,65,SIMPLE ROBBERY,1B,NaN,NaN,NaN,3669570,518651,2012-01-02 09:12:00,...,NaN,NaN,009XX Bordeaux St,70115.0,2,"(29.92046691792707, -90.1061762712039)",VIOLENT CRIME,SIMPLE ROBBERY,29.92046691792707,-90.1061762712039
4,A0248912,65,SIMPLE ROBBERY,1B,NaN,NaN,NaN,3692359,543219,2012-01-02 17:25:00,...,NaN,NaN,030XX Morrice Duncan Dr,70126.0,5,"(29.987329613283705, -90.03336254702)",VIOLENT CRIME,SIMPLE ROBBERY,29.987329613283705,-90.03336254702


In [41]:
cfs_final.tail()

,NOPD_Item,Type_,TypeText,Priority,InitialType,InitialTypeText,InitialPriority,MapX,MapY,TimeCreate,...,SelfInitiated,Beat,BLOCK_ADDRESS,Zip,PoliceDistrict,Location,CrimeType,Description,Latitude,Longitude
13922,J1221215,42B,ORAL SEXUAL BATTERY,2A,42B,ORAL SEXUAL BATTERY,1A,37369000,3513814,2015-10-10 16:58:22,...,N,4C03,Amazon St & Kansas St,70114.0,4,"(0, 0)",VIOLENT CRIME,ORAL SEXUAL BATTERY,0,0
13923,K0073015,42B,ORAL SEXUAL BATTERY,2A,21,COMPLAINT OTHER,2A,37369000,3513814,2015-11-01 15:14:57,...,N,3U02,050XX Chef Menteur Hwy,70126.0,3,"(0, 0)",VIOLENT CRIME,ORAL SEXUAL BATTERY,0,0
13924,K3038915,42B,ORAL SEXUAL BATTERY,1A,42B,ORAL SEXUAL BATTERY,1A,37369000,3513814,2015-11-26 17:29:18,...,N,3I01,008XX Louque Pl,70124.0,3,"(0, 0)",VIOLENT CRIME,ORAL SEXUAL BATTERY,0,0
13925,C0577416,42B,ORAL SEXUAL BATTERY,2A,21,COMPLAINT OTHER,1H,37369000,3513814,2016-03-05 15:46:45,...,N,3M03,015XX Milton St,70122.0,3,"(0, 0)",VIOLENT CRIME,ORAL SEXUAL BATTERY,0,0
13926,C1635316,42B,ORAL SEXUAL BATTERY,2A,21,COMPLAINT OTHER,1H,37369000,3513814,2016-03-14 13:12:14,...,N,3S02,020XX Lakeshore Dr,70122.0,3,"(0, 0)",VIOLENT CRIME,ORAL SEXUAL BATTERY,0,0


# 6. Check Beat Values

In [42]:
# Count of Null Beat records
len(cfs_final[cfs_final.Beat.isnull()])

4855

## A. Load missing Beat Values

In [43]:
filename = "../Datasets/Raw_Data/Shared_by_NOPD/2012 and 2013 CFS Item Number and Beat.csv"

In [44]:
df_beat = pd.read_csv(filename, header=None)

# Rename columns
df_beat.rename(columns={0:'NOPD_Item',1:'Beat'},inplace=True)

In [45]:
df_beat.head()

,NOPD_Item,Beat
0,A0000112,8C04
1,A0000113,5E01
2,A0000212,5P03
3,A0000213,4E04
4,A0000312,6I02


In [76]:
NOPD_Item_list = list(cfs_final.NOPD_Item)

In [80]:
df_beat = df_beat[df_beat.NOPD_Item.isin(NOPD_Item_list)]

In [91]:
df_beat = df_beat[df_beat.Beat.notnull()]

In [98]:
# Fill missing Beat Values
for row in cfs_final[cfs_final.Beat.isnull()].iterrows():
    if(row[1].NOPD_Item in list(df_beat.NOPD_Item)):
        cfs_final['Beat'].loc[cfs_final.NOPD_Item == row[1].NOPD_Item] = \
                    df_beat.Beat[df_beat.NOPD_Item == row[1].NOPD_Item].values[0]

In [99]:
# Count of Null Beat records
len(cfs_final[cfs_final.Beat.isnull()])

8

In [100]:
# % of records with Null beat values
len(cfs_final[cfs_final.Beat.isnull()])*100.0/len(cfs_final)

0.05744237811445394

In [101]:
len(cfs_final)

13927

# 7. Save CFS Data to csv

In [102]:
# Save the data
fullpath = "..\\Datasets\\Final_Data\\Calls_for_Service_Cleaned.csv"
cfs_final.to_csv(fullpath, sep=',',  index = False)

# 8. Zip with most Violent Crimes

In [103]:
zip_crime_count = Counter(cfs_final.Zip)

In [104]:
zip_crime_count.most_common(5)

[(70119.0, 1603),
 (70117.0, 1518),
 (70126.0, 1315),
 (70116.0, 1291),
 (70127.0, 906)]